# 19 卷积层

## 多层感知机的限制

无法获得图像的空间结构信息

全连接层等价于：

$|H|_{i,j} = |U|_{i,j} + \sum_k{\sum_l{|W|_{i,j,k,l}|X|_{k,l}}}$

$ = \sum_a{\sum_b{|V|_{i,j,a,b}|X|_{a,b}}} $

|W|的参数量十分大

## 不变性

### 平移不变性

不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”。

$$[\mathbf{H}]_{i, j} = u + \sum_a\sum_b [\mathbf{V}]_{a, b} [\mathbf{X}]_{i+a, j+b}.$$

### 局部性

神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测。

$$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}.$$



## 通道

图像可能有色彩信息，因此单个像素还会有多个通道

$$[\mathsf{H}]_{i,j,d} = \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \sum_c [\mathsf{V}]_{a, b, c, d} [\mathsf{X}]_{i+a, j+b, c},$$

## 互相关运算

即一个卷积核对和它一样大小的图像区域的加权求和运算

In [48]:
import torch
from torch import nn

def corr2d(X, K):
    h = K.shape[0]
    w = K.shape[1]
    res = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            res[i,j] = (X[i:i+h,j:j+w]*K).sum()
            # 注意这里tensor矩阵的下标表示为[,]而非[][]
    return res

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 卷积层

In [49]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self,X):
        return corr2d(X,self.weight)+self.bias


## 图像中目标的边缘检测

首先，我们构造一个6*8像素的黑白图像。中间四列为黑色(0)，其余像素为白色(1)

In [50]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [51]:
K = torch.tensor([[-1,1]]) # 注意这里两层中括号，高度为1，宽度为2

In [52]:
Y = corr2d(X, K)
Y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [53]:
corr2d(X.t(), K) # X转置后1和-1都消失了，证明这个卷积层只能检测垂直边缘

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习卷积核

In [54]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

conv2d.weight.data.reshape((1, 2))

epoch 2, loss 4.204
epoch 4, loss 0.708
epoch 6, loss 0.120
epoch 8, loss 0.020
epoch 10, loss 0.004


tensor([[-0.9877,  0.9910]])

## 填充和步幅

### 填充：

如果用上述卷积步骤，每次卷积会使图片小一圈；这时可以在原图像周围填充一圈（一般填充0）以使得输出图像不变小

![](./assets/conv-pad.svg)

In [55]:
import torch
from torch import nn

# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1) # 输入通道和输出通道均为1,卷积核为3×3,填充为1
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

对长和宽不同的kernel_size, 可以改变padding的长宽，使输出和输入的长宽一致

In [56]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

如果想要输入和输出形状相同,核形状应该满足pading*2+1

### 步幅

分为两个维度：垂直方向步幅和水平方向步幅

每次卷积核在对应方向移动的格数

In [57]:
print(X.shape)
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])


torch.Size([4, 4])

In [58]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

## 多输入输出通道

### 多输入通道

对每个输入通道执行互相关操作，对相同位置上的不同通道相加

In [59]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K))

X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]],
                  [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
K = torch.tensor([[[0.0,1.0],[2.0,3.0]],[[1.0,2.0],[3.0,4.0]]])

print(corr2d_multi_in(X, K))

tensor([[ 56.,  72.],
        [104., 120.]])


### 多输出通道

